In [ ]:
import sys
import os
from env.env1D import env1D
import filepath as file
import random #nur zum testen
import neat
import pybullet as p
import visualize
import graphviz
import time
import math
import numpy as np
#genomes, config
def getGoalPosition(upperRad, lowerRad, upperTheta, lowerTheta, upperPhi, lowerPhi, num): #Minimum Radius = 0.465, Höhe Gelenk = 0.36, MaxRadius = 0.921
    goalPos = []
    for i in range(num):
        rad = random.uniform(lowerRad, upperRad)
        theta = random.uniform(lowerTheta, upperTheta)
        #phi = random.uniform(lowerPhi, upperPhi)
        phi = 90 # x-Koordinate = 0
        zOffset = 0.36 #Höhe des Gelenkes
        x = math.sin(math.radians(theta)) * math.cos(math.radians(phi)) * rad
        y = math.sin(math.radians(theta)) * math.sin(math.radians(phi)) * rad
        z = math.cos(math.radians(theta)) * rad + zOffset
        goal = [x,y,z]
        goalPos.append(goal)
        
    print(goalPos)
    return goalPos

def evalFitness(isState, basePos, goalPos):
    x = isState[0] - basePos[0]   #Könnte ich auch theoretisch weglassen, aktuell noch zur Übersicht drin
    y = isState[1] - basePos[1]
    z = isState[2] - basePos[2]
    goalX = goalPos[0]
    goalY = goalPos[1]
    goalZ = goalPos[2]
    #print('aktueller Winkel: ', isAngle )
    distance = math.sqrt(((goalZ-z)**2)+((goalY-y)**2)+((goalX-x)**2))
    maxDistance = 2
    score = (maxDistance - distance)/maxDistance * 100
    return score

def getStartPos(num):
    pos = []
    rows = num/2
    for i in range(num):
        pos.append([3*(i%5), (i/5)*3, 0])
    return pos
        
def evalGenome(genomes,config):
    numOfPoints = 5    
    maxStep=2000
    ge = []
    nets = []
    bots = []
    #goalAngle = 100
    goal = getGoalPosition(0.921, 0.467, 90, -90, 169, -169, numOfPoints)
    startPos = getStartPos(len(genomes))
    
    #p.connect(p.DIRECT) #ohne GUI
    p.connect(p.GUI) #Mit GUI
    index = 0
    for genome_id, genome in genomes:
        net = neat.nn.FeedForwardNetwork.create(genome,config)
        nets.append(net)
        ge.append(genome)
        start = startPos[index]
        bots.append(env1D(start))
        genome.fitness = 0
        index += 1
    maxDistance = p.getLinkState(bots[0].botId,6)[0][2] - p.getLinkState(bots[0].botId,0)[0][2]
    print(maxDistance)
    index = 0
    stepCount = 0
    step = 0
    while(step < numOfPoints):
        while(stepCount<maxStep):
            for i, bot in enumerate(bots):
                output = nets[i].activate((bot.jointPosition[1],
                                           bot.jointPosition[3],
                                           p.getLinkState(bot.botId,6)[0][1]-bot.startPosition[1],
                                           p.getLinkState(bot.botId,6)[0][2],
                                           goal[step][1],
                                           goal[step][2]))
                if len(p.getContactPoints(bodyA=bot.botId, bodyB=bot.botId)) > 0:   #Kollision Roboter mit sich selbst
                    bot.collisionFlag = True
                if len(p.getContactPoints(bodyA=bot.botId, bodyB=bot.planeId)) > 0: #Kollision Roboter Bodenplatte
                    bot.collisionFlag = True
                if output[0] > 0 and output[1] < 0:
                    bot.moveDir[1] = -1
                    bot.moveFlag = True
                elif output[1] > 0 and output[0] < 0:
                    bot.moveDir[1] = 1
                    bot.moveFlag = True
                else:
                    bot.moveDir[1] = 0
                if output[2] > 0 and output[3] < 0:
                    bot.moveDir[3] = -1
                    bot.moveFlag = True
                elif output[3] > 0 and output[2] < 0:
                    bot.moveDir[3] = 1
                    bot.moveFlag = True
                else:
                    bot.moveDir[3] = 0
                bot.moveStep()            
            p.stepSimulation()
            time.sleep(1/240)
            stepCount += 1
        stepCount = 0
        print('Step', step)
        for genome_id, genome in genomes:
            genome.fitness += evalFitness(p.getLinkState(bots[index].botId,6)[0], bots[index].startPosition, goal[step])
            index += 1
        step += 1
        index = 0


    for genome_id, genome in genomes:
        genome.fitness = genome.fitness/numOfPoints
        if not bots[index].moveFlag:
            genome.fitness = -1
        if bots[index].moveDir[1] != 0:
            genome.fitness = 0
        if bots[index].moveDir[3] != 0:
            genome.fitness = 0
        if bots[index].collisionFlag:
            genome.fitness = 0
        index += 1
        print('Fitness:', genome.fitness)
    index = 0

    p.disconnect() 
        
        


In [ ]:


local_dir = file.getPath()
config_path = os.path.join(local_dir, 'config.txt')
try:
    import cPickle as pickle  # pylint: disable=import-error
except ImportError:
    import pickle  # pylint: disable=import-error
global pop
config = neat.config.Config(
    neat.DefaultGenome,
    neat.DefaultReproduction,
    neat.DefaultSpeciesSet,
    neat.DefaultStagnation,
    config_path
)
pop = neat.Population(config)
pop.add_reporter(neat.StdOutReporter(True))
stats = neat.StatisticsReporter()
pop.add_reporter(stats)
#pop.add_reporter(neat.Checkpointer(5))
winner = pop.run(evalGenome, 200)
    # Save the winner.
with open('winner-ctrnn', 'wb') as f:
    pickle.dump(winner, f)

print(winner)

visualize.plot_stats(stats, ylog=True, view=True, filename="ctrnn-fitness.svg")
visualize.plot_species(stats, view=True, filename="ctrnn-speciation.svg")

node_names = {-1: 'Joint1', -2: 'Joint3', -3: 'Y-Roboter', -4: 'Z-Roboter', -5: 'Y-Ziel', -6: 'Z-Ziel' 0: 'Joint1_1', 1: 'Joint1_2', 2:'Joint3_1', 3:'Joint3_2'}
visualize.draw_net(config, winner, True, node_names=node_names)

visualize.draw_net(config, winner, view=True, node_names=node_names,
                       filename="winner-ctrnn.gv")
visualize.draw_net(config, winner, view=True, node_names=node_names,
                       filename="winner-ctrnn-enabled.gv", show_disabled=False)


In [ ]:
#TODO Testumgebung
try:
    import cPickle as pickle  # pylint: disable=import-error
except ImportError:
    import pickle  # pylint: disable=import-error
with open('winner-ctrnn', "rb") as f:
    genome = pickle.load(f)
  
net = neat.nn.FeedForwardNetwork.create(genome,config)
goal = [90, -90, 30, -150] #Zielpunkte eingeben
maxStep = 2000
step = 0
stepCount = 0
bot = env1D()
while(step < len(goal)):
        while(stepCount<maxStep):
            output = net.activate((bot.jointPosition[0],(goal[step]/180)*math.pi))
            if len(p.getContactPoints(bodyA=bot.botId, bodyB=bot.botId)) > 0:
                 bot.collisionFlag = True
            if output[0] > 0 and output[1] < 0:
                bot.moveDir[0] = -1
                bot.moveFlag = True
            elif output[1] > 0 and output[0] < 0:
                bot.moveDir[0] = 1
                bot.moveFlag = True
            else:
                bot.moveDir[0] = 0
            bot.moveStep()            
            p.stepSimulation()
            time.sleep(1/240)
            stepCount += 1
        stepCount = 0
        #Calculate Difference between goal and actual point
        step += 1

In [ ]:
#TO DO: 
#1.Multiple Robots in the same simulation, write in .py File, maybe problems with Jupyter Lab and p.connect(p.Direct) DONE
#2.Rework the fitness function DONE
#3.Play around with the config file
#4.Collision Detection DONE
#